# Practical training - Variational Method
## Testing the gradient

the gradient itself can be tested, using the Taylor formula
$$J(\mathbf{x}+\mathbf{h})=J(\mathbf{x})+\left<\nabla_{\mathbf{x}}J,\mathbf{h}\right>+o(\|\mathbf{h}\|) \qquad \forall {\bf h}\in \mathbb{R}^n$$

So, in particular for $\mathbf{h}=\varepsilon\nabla_{\mathbf{x}}J$

$$\lim_{\varepsilon\rightarrow 0}\frac{J(\mathbf{x}+\varepsilon\nabla_{\mathbf{x}}J) - J(\mathbf{x})}{\varepsilon\|\nabla_{\mathbf{x}}J\|^2}=1$$


To do so, lets import the necessary modules

In [ ]:
from burgers import *
from gausscov import *
from simvar import *
from obsopt import *
import math


And initialise the assimilation parameters and matrices

In [ ]:
# Space-time domain
nx = 40                     # number of grid points
dx = 1./nx                  # space step
xx = np.array(range(nx))*dx # grid points abscissa
dt = 0.5*dx                 # time step
nt = 20                     # number of time steps
ns = 0                      # numerical scheme

M=Burgers(nx,dx,dt,ns)



In [ ]:
# Error staristics
sigmab = 0.01              # background state error std
sigmao = 0.001             # Observation error std
Lb = 0.05                  # Correlation length for B matrix

# Assimilation Parameters

precond = True              # preconditioning by square root of B 
iobstsub = 5                # Frequency of temporal subsampling of observations, [1:nt], 1=every time step
iobsxsub = 4                # Frequency of spatial subsampling of observations, [1:nx], 1=every space step

# Observation operator and error covariance matrix

H = Obsopt(nx,iobsxsub,nt,iobstsub)
R = sigmao*sigmao*np.eye(H.nobs,H.nobs)



In [ ]:
# Initialization of true field uo
uo=np.sin(2*math.pi*xx);
true=H.gen_obs(M,uo,sigmao)

# Initialization of background
ub=np.cos(2*math.pi*xx)
ubkg=[ub]
for it in range(nt):
    ub=M.step(ub)
    ubkg.append(ub)

In [ ]:
# Initialization of B matrix and its inverse

if precond:
    indic=2
else:
    indic=1
    
B=gausscov(nx,sigmab,Lb,indic)

var=Variational(ubkg[0],nt,B,M,H,R,precond)


So that we can compute $J(\mathbf{x})$ and $\nabla_{\mathbf{x}}J$ for a random $\mathbf{x}$

In [ ]:
uopt= np.random.normal(0.,sigmab,uo.size)

Jini  = var.cost(uopt)
grini = var.grad(uopt)
norm  = grini.dot(grini)

And finally look at what happens when $\alpha$ tends to 0

In [ ]:
alpha=0.001

for iii in range(1,21):
    uctl = uopt + alpha * grini
    J = var.cost(uctl)
    print ('1e-'+str(iii+2).zfill(2), (J-Jini)/(alpha*norm))
    alpha /= 10.
